For running over provided test samples

In [ ]:
import numpy 
import os
import pandas as pd
import librosa
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import seaborn as sns
from tensorflow.keras.models import load_model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import csv
temp = {'fileid':os.listdir('/content/drive/MyDrive/Audio_Classification-MLSP/Audio_Classification-MLSP-test')}

In [ ]:
temp

{'fileid': ['MLSP_test1379148.npy',
  'MLSP_test1865154.npy',
  'MLSP_test2182439.npy',
  'MLSP_test2471429.npy',
  'MLSP_test2082223.npy',
  'MLSP_test2142247.npy',
  'MLSP_test1458291.npy',
  'MLSP_test2336133.npy',
  'MLSP_test1235269.npy',
  'MLSP_test2211271.npy',
  'MLSP_test1053175.npy',
  'MLSP_test1116162.npy',
  'MLSP_test2406.npy',
  'MLSP_test149028.npy',
  'MLSP_test140846.npy',
  'MLSP_test1289164.npy',
  'MLSP_test2065366.npy',
  'MLSP_test1810394.npy',
  'MLSP_test2115430.npy',
  'MLSP_test1683220.npy',
  'MLSP_test130220.npy',
  'MLSP_test2012248.npy',
  'MLSP_test20111.npy',
  'MLSP_test1194176.npy',
  'MLSP_test2098444.npy',
  'MLSP_test2158293.npy',
  'MLSP_test1216256.npy',
  'MLSP_test1071.npy',
  'MLSP_test132489.npy',
  'MLSP_test2062232.npy',
  'MLSP_test213499.npy',
  'MLSP_test1362223.npy',
  'MLSP_test113179.npy',
  'MLSP_test2090121.npy',
  'MLSP_test173438.npy',
  'MLSP_test1516233.npy',
  'MLSP_test1515407.npy',
  'MLSP_test1968494.npy',
  'MLSP_test20548

In [ ]:
#List containing all spectrograms
mel_spectrograms=[] 

for file in temp['fileid']:
  arr=numpy.load(os.path.join('/content/drive/MyDrive/Audio_Classification-MLSP/Audio_Classification-MLSP-test', file))
  m,n,o=arr.shape
  print(m, n, o)
  arr.resize(n,o)
  print(arr.shape)
  # Convert a power spectrogram (amplitude squared) to decibel (dB) units
  mel_spectrogram = librosa.power_to_db(arr, ref=numpy.max)
  mel_spectrograms.append(mel_spectrogram)

1 128 680
(128, 680)
1 128 87
(128, 87)
1 128 71
(128, 71)
1 128 784
(128, 784)
1 128 345
(128, 345)
1 128 258
(128, 258)
1 128 856
(128, 856)
1 128 1135
(128, 1135)
1 128 74
(128, 74)
1 128 293
(128, 293)
1 128 1726
(128, 1726)
1 128 431
(128, 431)
1 128 345
(128, 345)
1 128 2479
(128, 2479)
1 128 465
(128, 465)
1 128 563
(128, 563)
1 128 150
(128, 150)
1 128 67
(128, 67)
1 128 256
(128, 256)
1 128 79
(128, 79)
1 128 587
(128, 587)
1 128 2369
(128, 2369)
1 128 1212
(128, 1212)
1 128 2154
(128, 2154)
1 128 1523
(128, 1523)
1 128 1412
(128, 1412)
1 128 431
(128, 431)
1 128 345
(128, 345)
1 128 89
(128, 89)
1 128 1103
(128, 1103)
1 128 133
(128, 133)
1 128 1433
(128, 1433)
1 128 1654
(128, 1654)
1 128 1383
(128, 1383)
1 128 913
(128, 913)
1 128 48
(128, 48)
1 128 982
(128, 982)
1 128 160
(128, 160)
1 128 352
(128, 352)
1 128 237
(128, 237)
1 128 1648
(128, 1648)
1 128 345
(128, 345)
1 128 1969
(128, 1969)
1 128 701
(128, 701)
1 128 634
(128, 634)
1 128 2209
(128, 2209)
1 128 1091
(128, 1

In [ ]:
# padded according to max_duration of time used while training the model
max_duration = 2584

# add padding in the given .npy files
features=[]
for spec in mel_spectrograms:
  mat=numpy.pad(spec, [(0, 0), (0, max_duration-spec[0].size)], mode='constant', constant_values=0)
  mat=mat.reshape((mat.shape[0], mat.shape[1], 1))
  features.append(mat)

In [ ]:
# convert features list into numpy.ndarray type
features=numpy.array(features)

In [ ]:
# need to convert training data into tensor datatype
i=0
for _ in features:
    features[i]= tf.convert_to_tensor(features[i])
    i=i+1

In [ ]:
X=features

In [ ]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

train_encoder = LabelEncoder()
train_ = train_encoder.fit_transform(y_train)
to_onehot=to_categorical(train_)
y_train=to_onehot

test_encoder = LabelEncoder()
test_ = test_encoder.fit_transform(y_test)
to_onehot=to_categorical(test_)
y_test=to_onehot

In [ ]:
# reference: https://datascience.stackexchange.com/questions/45165/how-to-get-accuracy-f1-precision-and-recall-for-a-keras-model
from keras import backend 
from sklearn.metrics import precision_score , recall_score
def recall_m(y_true, y_pred):
    true_positives = backend.sum(backend.round(backend.clip(y_true * y_pred, 0, 1)))
    possible_positives = backend.sum(backend.round(backend.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + backend.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = backend.sum(backend.round(backend.clip(y_true * y_pred, 0, 1)))
    predicted_positives = backend.sum(backend.round(backend.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + backend.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+backend.epsilon())) 

In [ ]:
filepath = '/content/drive/MyDrive/Audio_Classification-MLSP/my_best_model_cnn.hdf5'
model = load_model(filepath, custom_objects={'f1_m':f1_m, 'precision_m':precision_m, 'recall_m':recall_m})

In [ ]:
predictions = model.predict(X)

In [ ]:
predictions_decoded = []
for i in predictions:
  predictions_decoded.append(numpy.argmax(i))

In [ ]:
print(predictions_decoded)

[5,
 6,
 8,
 9,
 7,
 8,
 2,
 9,
 4,
 8,
 9,
 4,
 7,
 3,
 2,
 4,
 7,
 6,
 8,
 6,
 1,
 6,
 0,
 4,
 7,
 4,
 4,
 8,
 0,
 8,
 8,
 5,
 0,
 7,
 6,
 5,
 5,
 6,
 7,
 4,
 8,
 7,
 5,
 4,
 3,
 3,
 8,
 6,
 5,
 4,
 8,
 3,
 6,
 4,
 6,
 4,
 6,
 6,
 9,
 5,
 3,
 4,
 1,
 4,
 8,
 9,
 8,
 5,
 7,
 7,
 5,
 6,
 0,
 9,
 4,
 8,
 6,
 8,
 6,
 6,
 1,
 5,
 1,
 8,
 1,
 9,
 8,
 6,
 6,
 4,
 8,
 8,
 4,
 8,
 8,
 9,
 4,
 5,
 1,
 0,
 5,
 5,
 1,
 8,
 9,
 6,
 0,
 1,
 8,
 3,
 2,
 1,
 2,
 3,
 8,
 2,
 3,
 3,
 2,
 1,
 1,
 8,
 1,
 0,
 8,
 3,
 1,
 0,
 3,
 9,
 3,
 9,
 8,
 2,
 3,
 0,
 9,
 7,
 0,
 0,
 0,
 0,
 2,
 3,
 3,
 1,
 4,
 3,
 3,
 0,
 9,
 0,
 0,
 3,
 8,
 3,
 9,
 3,
 8,
 1,
 0,
 8,
 3,
 2,
 2,
 0,
 1,
 7,
 2,
 2,
 0,
 3,
 8,
 5,
 2,
 1,
 3,
 3,
 0,
 3,
 3,
 2,
 1,
 0,
 2,
 2,
 3,
 0,
 9,
 2,
 9,
 1,
 1,
 8,
 1,
 2,
 9,
 2,
 9,
 0,
 1]

In [ ]:
# model used for testing
decode = {0:'Bark', 9:'Walk_and_footsteps', 2:'Doorbell', 1:'Crying_and_sobbing', 7:'Siren', 3:'Knock', 8:'Vehicle_horn_and_car_horn_and_honking', 5:'Microwave_oven', 6:'Shatter', 4:'Meow'}

In [ ]:
predictions_string = []
for i in predictions_decoded:
  predictions_string.append(decode[i])

In [ ]:
print(predictions_string)

['Microwave_oven', 'Shatter', 'Vehicle_horn_and_car_horn_and_honking', 'Walk_and_footsteps', 'Siren', 'Vehicle_horn_and_car_horn_and_honking', 'Doorbell', 'Walk_and_footsteps', 'Meow', 'Vehicle_horn_and_car_horn_and_honking', 'Walk_and_footsteps', 'Meow', 'Siren', 'Knock', 'Doorbell', 'Meow', 'Siren', 'Shatter', 'Vehicle_horn_and_car_horn_and_honking', 'Shatter', 'Crying_and_sobbing', 'Shatter', 'Bark', 'Meow', 'Siren', 'Meow', 'Meow', 'Vehicle_horn_and_car_horn_and_honking', 'Bark', 'Vehicle_horn_and_car_horn_and_honking', 'Vehicle_horn_and_car_horn_and_honking', 'Microwave_oven', 'Bark', 'Siren', 'Shatter', 'Microwave_oven', 'Microwave_oven', 'Shatter', 'Siren', 'Meow', 'Vehicle_horn_and_car_horn_and_honking', 'Siren', 'Microwave_oven', 'Meow', 'Knock', 'Knock', 'Vehicle_horn_and_car_horn_and_honking', 'Shatter', 'Microwave_oven', 'Meow', 'Vehicle_horn_and_car_horn_and_honking', 'Knock', 'Shatter', 'Meow', 'Shatter', 'Meow', 'Shatter', 'Shatter', 'Walk_and_footsteps', 'Microwave_oven

In [ ]:
temp['prediction'] = predictions_string

In [ ]:
print(temp)

{'fileid': ['MLSP_test1379148.npy', 'MLSP_test1865154.npy', 'MLSP_test2182439.npy', 'MLSP_test2471429.npy', 'MLSP_test2082223.npy', 'MLSP_test2142247.npy', 'MLSP_test1458291.npy', 'MLSP_test2336133.npy', 'MLSP_test1235269.npy', 'MLSP_test2211271.npy', 'MLSP_test1053175.npy', 'MLSP_test1116162.npy', 'MLSP_test2406.npy', 'MLSP_test149028.npy', 'MLSP_test140846.npy', 'MLSP_test1289164.npy', 'MLSP_test2065366.npy', 'MLSP_test1810394.npy', 'MLSP_test2115430.npy', 'MLSP_test1683220.npy', 'MLSP_test130220.npy', 'MLSP_test2012248.npy', 'MLSP_test20111.npy', 'MLSP_test1194176.npy', 'MLSP_test2098444.npy', 'MLSP_test2158293.npy', 'MLSP_test1216256.npy', 'MLSP_test1071.npy', 'MLSP_test132489.npy', 'MLSP_test2062232.npy', 'MLSP_test213499.npy', 'MLSP_test1362223.npy', 'MLSP_test113179.npy', 'MLSP_test2090121.npy', 'MLSP_test173438.npy', 'MLSP_test1516233.npy', 'MLSP_test1515407.npy', 'MLSP_test1968494.npy', 'MLSP_test205489.npy', 'MLSP_test1301172.npy', 'MLSP_test2206272.npy', 'MLSP_test2409.npy',

In [ ]:
df = pd.DataFrame(temp)
df

,fileid,prediction
0,MLSP_test1379148.npy,Microwave_oven
1,MLSP_test1865154.npy,Shatter
2,MLSP_test2182439.npy,Vehicle_horn_and_car_horn_and_honking
3,MLSP_test2471429.npy,Walk_and_footsteps
4,MLSP_test2082223.npy,Siren
...,...,...
196,MLSP_test2552281.npy,Walk_and_footsteps
197,MLSP_test684486.npy,Doorbell
198,MLSP_test2566454.npy,Walk_and_footsteps
199,MLSP_test68320.npy,Bark


In [ ]:
df.to_csv('cnn_predicitons.csv')